# Scenario-Based Tutorial: A Simple Control Loop

This notebook is based on the runnable scenario found in `scenarios/case_simple_control_loop/`. It demonstrates a more agent-centric way of defining and running a simulation, using a YAML configuration file to define an **"agent society"**.

## 1. Goal of the Scenario

The goal is to maintain a water level of **10.0m** in a tank. 

This scenario is different from previous examples because it assembles a complete society of specialized agents that communicate via a message bus:

- An **InflowAgent** reads inflow data from a CSV file and publishes it.
- A **TankAgent** represents the physical tank. It subscribes to the inflow data and to commands for its release valve. It publishes its own state (e.g., water level).
- A **PIDAgent** subscribes to the tank's state, compares it to a setpoint, and publishes a command for the valve.
- A **ValveAgent** subscribes to the PID's commands and simulates a physical valve, which affects the tank's outflow.
- A **DataLoggerAgent** subscribes to various topics to log the results to a file.

## 2. The Configuration File (`config.yaml`)

The entire society of agents and the simulation parameters are defined in a single YAML file. This demonstrates a powerful, declarative way to build complex systems.

In [ ]:
# We will use the pyyaml library to load and display the config
import yaml

config_path = '../scenarios/case_simple_control_loop/config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Display the loaded configuration
print(yaml.dump(config, sort_keys=False))

## 3. The Runner Script (`run.py`)

The Python script to run this scenario is very simple. It uses a `Launcher` class, which is a high-level utility that handles:
1. Parsing the YAML file.
2. Creating an `AgentKernel`.
3. Instantiating all the agents defined in the `agent_society` section.
4. Running the simulation.

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from chs_sdk.core.launcher import Launcher

# --- Execute the Simulation ---
print("Starting simulation...")
launcher = Launcher()
launcher.run(config)
print("Simulation complete.")

# --- Load and Plot Results ---
# The DataLoggerAgent saved the results to 'simulation_results.csv'
results_df = pd.read_csv('simulation_results.csv')

print("\nFirst 5 rows of results:")
print(results_df.head())

## 4. Visualization

Let's plot the tank's water level and the valve command signal from the results file.

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Water Level
color = 'tab:blue'
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Water Level (m)', color=color)
# The column name is a bit complex, it's how the logger names it
level_column = "('main_tank', 'level')"
ax1.plot(results_df['time'], results_df[level_column], color=color, label='Water Level')
ax1.axhline(y=10.0, color='r', linestyle='--', label='Setpoint')
ax1.tick_params(axis='y', labelcolor=color)

# Plot Valve Command on a second y-axis
ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Valve Command', color=color)
command_column = "('pid_controller', 'output')"
ax2.plot(results_df['time'], results_df[command_column], color=color, linestyle=':', alpha=0.7, label='Valve Command')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Agent-Based PID Control Scenario')
fig.tight_layout()
# Manually create legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)
plt.grid(True)
plt.show()

This notebook demonstrates a powerful, declarative, and agent-oriented approach to building simulations. Instead of manually connecting components, we define a society of agents that communicate over a message bus, which is a very scalable and flexible architecture.